# VSCode Colab Runner: Local Repo + Colab Compute (CosyVoice)

This notebook is designed for the VSCode Colab extension. It runs your websocket stack on Colab GPU while keeping repo state reproducible via git.

Important:
- Colab cannot see uncommitted local files from your laptop. Push your local commit first, then set `GIT_REF` to that commit hash.
- Use a GPU runtime in Colab before running cells.


In [ ]:
import os
import textwrap

# Set these before running if needed.
REPO_URL = os.environ.get("REPO_URL", "https://github.com/Aryan-Seth/sigiq_task.git")
GIT_REF = os.environ.get("GIT_REF", "main")  # branch, tag, or commit SHA

WORK_ROOT = os.environ.get("WORK_ROOT", "/content")
REPO_PARENT = os.environ.get("REPO_PARENT", f"{WORK_ROOT}/sigiq_takehome")
REPO_ROOT = os.environ.get("REPO_ROOT", f"{REPO_PARENT}/tts_ws")
COSYVOICE_REPO_DIR = os.environ.get("COSYVOICE_REPO_DIR", f"{WORK_ROOT}/CosyVoice")

COSYVOICE_MODEL_DIR = os.environ.get("COSYVOICE_MODEL_DIR", "FunAudioLLM/CosyVoice2-0.5B")
COSYVOICE_MODE = os.environ.get("COSYVOICE_MODE", "sft")
PORT = os.environ.get("PORT", "8000")
HOST = os.environ.get("HOST", "0.0.0.0")

os.environ.update({
    "REPO_URL": REPO_URL,
    "GIT_REF": GIT_REF,
    "WORK_ROOT": WORK_ROOT,
    "REPO_PARENT": REPO_PARENT,
    "REPO_ROOT": REPO_ROOT,
    "COSYVOICE_REPO_DIR": COSYVOICE_REPO_DIR,
    "COSYVOICE_MODEL_DIR": COSYVOICE_MODEL_DIR,
    "COSYVOICE_MODE": COSYVOICE_MODE,
    "PORT": PORT,
    "HOST": HOST,
})

print(textwrap.dedent(f"""
REPO_URL={REPO_URL}
GIT_REF={GIT_REF}
REPO_ROOT={REPO_ROOT}
COSYVOICE_REPO_DIR={COSYVOICE_REPO_DIR}
COSYVOICE_MODEL_DIR={COSYVOICE_MODEL_DIR}
COSYVOICE_MODE={COSYVOICE_MODE}
PORT={PORT}
"""))


In [ ]:
%%bash
set -euo pipefail
nvidia-smi || true

mkdir -p "$REPO_PARENT"
if [[ ! -d "$REPO_PARENT/.git" ]]; then
  git clone "$REPO_URL" "$REPO_PARENT"
fi

cd "$REPO_PARENT"
git fetch --all --prune
git checkout "$GIT_REF"

echo "Repo HEAD: $(git rev-parse --short HEAD)"
echo "Repo path: $REPO_ROOT"
ls -la "$REPO_ROOT" | head -n 40


In [ ]:
%%bash
set -euo pipefail
cd "$REPO_ROOT"

# Installs CosyVoice and runtime deps.
bash colab_t4_cosyvoice.sh install


In [ ]:
%%bash
set -euo pipefail
pkill -f "uvicorn app.server:app" >/dev/null 2>&1 || true

cd "$REPO_ROOT"
export TTS_BACKEND=cosyvoice
export COSYVOICE_REPO_DIR="$COSYVOICE_REPO_DIR"
export COSYVOICE_MODEL_DIR="$COSYVOICE_MODEL_DIR"
export COSYVOICE_MODE="$COSYVOICE_MODE"
export PORT="$PORT"
export HOST="$HOST"
export TTS_PROFILE=1

bash colab_t4_cosyvoice.sh start


In [ ]:
import asyncio
import base64
import json
import os
import wave

import websockets

PORT = int(os.environ.get("PORT", "8000"))
OUT_WAV = "/content/cosy_smoke.wav"

async def run_smoke() -> None:
    pcm_parts = []
    metrics = None
    async with websockets.connect(f"ws://127.0.0.1:{PORT}/tts", max_size=None) as ws:
        await ws.send(json.dumps({"text": " ", "flush": False, "run_id": "colab-smoke"}))
        await ws.send(json.dumps({"text": "Hello from VSCode Colab CosyVoice websocket.", "flush": False, "run_id": "colab-smoke"}))
        await ws.send(json.dumps({"text": "", "flush": True, "run_id": "colab-smoke"}))
        await ws.send(json.dumps({"text": "", "flush": False, "run_id": "colab-smoke"}))

        try:
            while True:
                msg = await ws.recv()
                payload = json.loads(msg if isinstance(msg, str) else msg.decode("utf-8"))
                if payload.get("type") == "metrics":
                    metrics = payload.get("metrics")
                    continue
                audio_b64 = str(payload.get("audio", ""))
                if audio_b64:
                    pcm_parts.append(base64.b64decode(audio_b64))
        except websockets.ConnectionClosed:
            pass

    pcm = b"".join(pcm_parts)
    with wave.open(OUT_WAV, "wb") as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(44100)
        wf.writeframes(pcm)

    print("audio bytes:", len(pcm))
    print("wrote:", OUT_WAV)
    print("server metrics:", metrics)

asyncio.run(run_smoke())


In [ ]:
%%bash
set -euo pipefail
cd "$REPO_ROOT"
python benchmark_ttft.py \
  --uri "ws://127.0.0.1:${PORT}/tts" \
  --backend cosyvoice \
  --no-start-server \
  --lengths 80,160,320 \
  --runs-per-length 2 \
  --chunk-mode ramp \
  --chunk-plan 4,8,32 \
  --delay 0.01 \
  --math-normalizer rule \
  --json-out /content/ttft_cosyvoice_colab.json

echo "Wrote /content/ttft_cosyvoice_colab.json"


In [ ]:
# Optional: expose UI publicly via Cloudflare quick tunnel
# (useful for testing the web UI from your laptop browser)
import os
import pathlib
import re
import subprocess
import time

port = int(os.environ.get("PORT", "8000"))
cf = pathlib.Path("/usr/local/bin/cloudflared")
log = pathlib.Path("/content/cloudflared.log")

if not cf.exists():
    subprocess.run(["wget", "-q", "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64", "-O", str(cf)], check=True)
    subprocess.run(["chmod", "+x", str(cf)], check=True)

subprocess.run(["pkill", "-f", "cloudflared tunnel"], check=False)
with log.open("w") as f:
    p = subprocess.Popen([str(cf), "tunnel", "--url", f"http://127.0.0.1:{port}"], stdout=f, stderr=subprocess.STDOUT)

url = None
for _ in range(40):
    time.sleep(0.5)
    if log.exists():
        txt = log.read_text(errors="ignore")
        m = re.search(r"https://[-a-z0-9]+\\.trycloudflare\\.com", txt)
        if m:
            url = m.group(0)
            break

print("Cloudflare URL:", url)
print("If URL is None, re-run this cell and check /content/cloudflared.log")
